In [12]:
import numpy as np
from qulacs import QuantumState, QuantumCircuit as QCQ, Observable
from qulacs.gate import RX, RZ, PauliRotation
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import RXGate, RZGate, RXXGate, RYYGate, RZZGate
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer import AerSimulator

nq = 2

obs_qiskit = SparsePauliOp.from_list([("IY", 1.0)])
qc_qiskit = QuantumCircuit(nq)

qc_qiskit.h(0)
qc_qiskit.h(1)
qc_qiskit.rz(np.pi / 4, 0)
qc_qiskit.rx(np.pi / 3, 1)
qc_qiskit.append(RXXGate(np.pi / 5), [0, 1])
qc_qiskit.append(RYYGate(np.pi / 3), [0, 1])
qc_qiskit.append(RZZGate(np.pi / 4), [0, 1])
qc_qiskit.save_expectation_value(obs_qiskit, list(range(nq)), "expval")
backend = AerSimulator(method="statevector")
result = backend.run(transpile(qc_qiskit, backend)).result()
qiskit_expectation = result.data(0)["expval"]
state_qulacs = QuantumState(nq)
qc_qulacs = QCQ(nq)
qc_qulacs.add_H_gate(0)
qc_qulacs.add_H_gate(1)
qc_qulacs.add_gate(RZ(0, np.pi / 4))
qc_qulacs.add_gate(RX(1, np.pi / 3))
qc_qulacs.add_gate(PauliRotation([0, 1], [1, 1], np.pi / 5))
qc_qulacs.add_gate(PauliRotation([0, 1], [2, 2], np.pi / 3))
qc_qulacs.add_gate(PauliRotation([0, 1], [3, 3], np.pi / 4))
qc_qulacs.update_quantum_state(state_qulacs)

# Observable: Y on qubit 0
obs_qulacs = Observable(nq)
obs_qulacs.add_operator(1.0, "Y 0")
qulacs_expectation = obs_qulacs.get_expectation_value(state_qulacs)
print(f"Qiskit  ⟨Y0⟩ = {qiskit_expectation:.6f}")
print(f"Qulacs  ⟨Y0⟩ = {qulacs_expectation:.6f}")

Qiskit  ⟨Y0⟩ = 0.404508
Qulacs  ⟨Y0⟩ = -0.404508
